In [1]:
from get_training_data import *

import tensorflow as tf
import keras
import pickle
import gzip
import os
import random
import matplotlib.pyplot as plt
%matplotlib nbagg

/home/andy/.local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
with gzip.open('train_cases.pkl.gz','rb') as fp:
    train_cases = pickle.load(fp)
type(train_cases), len(train_cases)

(tuple, 2)

In [3]:
train_X = train_cases[0]
train_y = train_cases[1]
ntrain = train_X.shape[0]
type(train_X), train_X.shape, train_X.dtype, type(train_y), train_y.shape, train_y.dtype

(numpy.ndarray,
 (16384, 16, 16, 128),
 dtype('float32'),
 numpy.ndarray,
 (16384, 1),
 dtype('float64'))

In [4]:
with gzip.open('valid_cases.pkl.gz','rb') as fp:
    valid_cases = pickle.load(fp)
valid_X = valid_cases[0]
valid_y = valid_cases[1]
nvalid = valid_X.shape[0]
type(valid_X), valid_X.shape, valid_X.dtype, type(valid_y), valid_y.shape, valid_y.dtype

(numpy.ndarray,
 (4096, 16, 16, 128),
 dtype('float32'),
 numpy.ndarray,
 (4096, 1),
 dtype('float64'))

In [5]:
def choose_training_case():
    i = random.randint(0, ntrain-1)
    return(train_X[i,:,:,:], train_y[i,:])

def choose_validation_case():
    i = random.randint(0, nvalid-1)
    return(valid_X[i,:,:,:], valid_y[i,:])

In [6]:
def make_batch_from_megabatch(batchsize=32, validation=False):
    get_case = choose_validation_case if validation else choose_training_case    
    xbatch = np.empty([0, INPUTS_PER_BEAT, NBEATS, NCHANNELS],dtype=np.float32)
    ybatch = np.empty([0, 1])
    for i in range(batchsize):
        case = get_case()
        x = np.expand_dims(case[0], axis=0)
        xbatch = np.concatenate([xbatch, x], axis=0)
        y = np.array(int(case[1])).reshape([1,1])
        ybatch = np.concatenate([ybatch, y], axis=0)
    return(xbatch, ybatch)

def train_gen(batchsize=32):
    while True:
        yield(make_batch_from_megabatch(batchsize, validation=False))

def valid_gen(batchsize=32):
    while True:
        yield(make_batch_from_megabatch(batchsize, validation=True))

In [7]:
# The generators are way too slow: have to fix this
for t in train_gen():
    break
t[0].shape, t[1].shape

((32, 16, 16, 128), (32, 1))

In [8]:
for v in valid_gen():
    break
v[0].shape, v[1].shape

((32, 16, 16, 128), (32, 1))

In [34]:
input_shape = (INPUTS_PER_BEAT, NBEATS, NCHANNELS)

model = keras.Sequential()
model.add(keras.layers.Conv2D(64, 1, activation='relu', input_shape=input_shape))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(32, 3, activation='relu'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(16, 5, activation='relu'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(8, 7, activation='relu'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Flatten())
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dropout(.3))
model.add(keras.layers.Dense(8, activation='relu'))
model.add(keras.layers.Dropout(.5))
model.add(keras.layers.Dense(1, activation='sigmoid'))

In [35]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [36]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 16, 16, 64)        8256      
_________________________________________________________________
batch_normalization_9 (Batch (None, 16, 16, 64)        256       
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 14, 14, 32)        18464     
_________________________________________________________________
batch_normalization_10 (Batc (None, 14, 14, 32)        128       
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 10, 10, 16)        12816     
_________________________________________________________________
batch_normalization_11 (Batc (None, 10, 10, 16)        64        
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 4, 4, 8)           6280      
__________

In [37]:
valid=valid_gen()
train=train_gen()
model.fit_generator(train, 
                    validation_data=valid, validation_steps=64,
                    epochs=40, 
                    steps_per_epoch=64)

Epoch 1/40
64/64 [==============================] - 2s 30ms/step - loss: 0.8402 - acc: 0.5820 - val_loss: 0.6207 - val_acc: 0.7065
Epoch 2/40
64/64 [==============================] - 1s 14ms/step - loss: 0.6346 - acc: 0.7002 - val_loss: 0.6429 - val_acc: 0.7476
Epoch 3/40
64/64 [==============================] - 1s 14ms/step - loss: 0.6268 - acc: 0.7339 - val_loss: 0.6025 - val_acc: 0.7500
Epoch 4/40
64/64 [==============================] - 1s 14ms/step - loss: 0.6077 - acc: 0.7407 - val_loss: 0.5895 - val_acc: 0.7510
Epoch 5/40
64/64 [==============================] - 1s 15ms/step - loss: 0.5971 - acc: 0.7559 - val_loss: 0.5688 - val_acc: 0.7637
Epoch 6/40
64/64 [==============================] - 1s 14ms/step - loss: 0.6010 - acc: 0.7568 - val_loss: 0.5687 - val_acc: 0.7671
Epoch 7/40
64/64 [==============================] - 1s 15ms/step - loss: 0.5882 - acc: 0.7554 - val_loss: 0.5643 - val_acc: 0.7646
Epoch 8/40
64/64 [==============================] - 1s 14ms/step - loss: 0.5535 - a

In [38]:
for i in range(10):
    song, tempo, compatible, clip = get_validation_case()
    print( song, tempo )
    c = clip_to_tf_input(resample_clip(clip))
    x = np.expand_dims(c, axis=0)
    p = model.predict(x)[0][0]
    print( 'Predicted: ', p, '   Actual: ', compatible, '\n' )

Kalamazoo 65
Predicted:  0.43960142    Actual:  True 

HangingOnTheTelephone 61
Predicted:  0.014644527    Actual:  False 

BloodyMerryMorning 43
Predicted:  0.31619284    Actual:  False 

IThinkWereAloneNow2MP 129
Predicted:  0.56190586    Actual:  True 

FugueForTinhorns2 41
Predicted:  0.32449898    Actual:  False 

HoldMeTight 91
Predicted:  0.071494624    Actual:  False 

53rdAnd3rdMP 287
Predicted:  0.28115976    Actual:  False 

HoldMeTight 136
Predicted:  0.5258412    Actual:  True 

TheBollWeevil 115
Predicted:  0.25462615    Actual:  False 

GoodGollyMissMollyWithJerryLeeAndFats 108
Predicted:  0.021275472    Actual:  False 

